# <font color='blue'>CMPE-297 Project Part B - Short Answer Extraction</font>

# Aaron Choi, Nelson Paz
### 1. Read CMU QA Corpus 
- QAanswerFiles.zip (found in Canvas)
- question_answer_pairs-combined.txt

#### Implement precision, recall and F1 score.
Rank all documents for a given query and return the top N documents (let's call this return set size)
Report your precision, recall and F1 for the whole CMU QA corpus for what you think is a good set of preprocessing steps and return set size.

In [39]:
# Importing Primary Libraries
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
import glob
import math
import os
from nltk.stem import WordNetLemmatizer
import json
import codecs
from nltk.tree import Tree
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import spacy
from heapq import nlargest 
from nltk.corpus import wordnet as wn
import string
import re
import pandas as pd
import numpy as np
nlp = spacy.load('en_core_web_lg')

# Extraction Functions 

In [5]:
def extracWords(text,fname=""):
    stopWords = set(stopwords.words('english'))
    words = []
    doc = nlp(text)
    for token in doc:
        words.append(token.text)
    nonStopSentence = [w for w in words if not w in stopWords] 
    return(nonStopSentence)

def extracLemma(text):
    lemmaWords =[]
    doc = nlp(text)
    for token in doc:
        lemmaWords.append(token.lemma_)
    return(lemmaWords)


In [52]:
# Relation Extraction From Wordnet
def extracRelations(wordScore):
    wordRelationsFeature = {}
    for key in wordScore:
        syn = wn.synsets(key)
        r_j = []
        if len(syn) != 0:
            for s in syn:
                synonyms = s.lemma_names()
                definition = s.definition()
                hyponyms = sorted([lemma.name() for synset in s.hyponyms() for lemma in synset.lemmas()])
                hypernyms = [lemma.name() for synset in  s.hypernyms() for lemma in synset.lemmas()]
                meronyms = s.part_meronyms()
                holonyms = s.part_holonyms()
                r_j.append({'synset':s,'relations':{'def':definition,'syn':synonyms,'hypo':hyponyms,'hyper':hypernyms,'mero':meronyms,'holo':holonyms}})
        else:
            wordRelationsFeature[key] = []
        wordRelationsFeature[key] = r_j         
    return wordRelationsFeature 

# Pulling all 150 documenst text.   

In [35]:
from nltk.tokenize import sent_tokenize
import time

path = input("Enter Path to the /QAanwerFiles folder" )  # Or Press ENTER if Path is edited below. 
if path == "":
    path = 'C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/' # The PATH to the Folder can be Edited HERE. 
ResutsPath = path
print(path)

Enter Path to the /QAanwerFiles folder
C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/


In [9]:
start = time.time()
MainDataText = []
MainCorpusLemma =[]
e = ['S10','S09','S08']

for X in range(0,3):
    SXX = str(e[X])
    path_A = path + SXX +'/data/'
    for i in range(1,(7-X)):
        path_B = (path_A+'set'+str(i)+'/')
        os.chdir(path_B)
        print(path_B)
        for filename in glob.glob("*.txt.clean"):
            sentences=[]
            doc = {}
            f = codecs.open(filename,'r','ISO-8859-1')
            file = f.read()
            print("Document processing -------> ",filename)   
            for l in sent_tokenize(file):
                if l.strip():
                    w_j_obj= extracWords(l,filename)
                    l_j_obj= extracLemma(l)
                    sentences.append({"sentence":l,"lemma":l_j_obj})
                    MainCorpusLemma.extend(l_j_obj)
            doc = {'doc':filename,'sentences':sentences}
            print("Number of Lines in document ----> ",len(sent_tokenize(file)))
            MainDataText.append(doc)

seconds = time.time() - start
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))

C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/S10/data/set1/
Document processing ------->  a1.txt.clean
Number of Lines in document ---->  394
Document processing ------->  a10.txt.clean
Number of Lines in document ---->  230
Document processing ------->  a2.txt.clean
Number of Lines in document ---->  276
Document processing ------->  a3.txt.clean
Number of Lines in document ---->  110
Document processing ------->  a4.txt.clean
Number of Lines in document ---->  254
Document processing ------->  a5.txt.clean
Number of Lines in document ---->  115
Document processing ------->  a6.txt.clean
Number of Lines in document ---->  345
Document processing ------->  a7.txt.clean
Number of Lines in document ---->  74
Document processing ------->  a8.txt.clean
Number of Lines in document ---->  54
Document processing ------->  a9.txt.clean
Number of Lines in document ---->  152
C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/S10/data/set2/
Document processing ------->  a1.txt.clea

Number of Lines in document ---->  119
Document processing ------->  a3.txt.clean
Number of Lines in document ---->  512
Document processing ------->  a4.txt.clean
Number of Lines in document ---->  246
Document processing ------->  a5.txt.clean
Number of Lines in document ---->  15
Document processing ------->  a6.txt.clean
Number of Lines in document ---->  41
Document processing ------->  a7.txt.clean
Number of Lines in document ---->  159
Document processing ------->  a8.txt.clean
Number of Lines in document ---->  56
Document processing ------->  a9.txt.clean
Number of Lines in document ---->  22
C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/S09/data/set5/
Document processing ------->  a1.txt.clean
Number of Lines in document ---->  302
Document processing ------->  a10.txt.clean
Number of Lines in document ---->  363
Document processing ------->  a2.txt.clean
Number of Lines in document ---->  385
Document processing ------->  a3.txt.clean
Number of Lines in document ---

'C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/'

In [38]:
# Saving the Pulling Data in the Main Folder. 

os.chdir(path)
try:
    os.mkdir('PullingResults')
except:
    print("Directory Already Exists")
    
with open('PullingResults/MainCorpus.json', 'w') as MainCopusFile:  
    json.dump(MainDataText, MainCopusFile)
print("Corpus File path -> "+path+'PullingResults/MainCorpus.json')

Corpus File path -> C:/Users/Admin/CMPE_297_NLP_FinalProject/QAanswerFiles/PullingResults/MainCorpus.json


In [50]:
# Leminized DataFrame of all Documents, 
data_df = pd.read_json(path+ 'PullingResults/MainCorpus.json', orient='records')   
from pandas import option_context
with option_context('display.max_colwidth', 200):
    display((data_df[:]))

,doc,sentences
0,a1.txt.clean,"[{'sentence': 'Ant Ants are social insects of the family Formicidae ( ), and along with the related wasps and bees, they belong to the order Hymenoptera.', 'lemma': ['ant', ' ', 'Ants', 'be'..."
1,a10.txt.clean,"[{'sentence': 'Octopus The octopus is a cephalopod of the order Octopoda.', 'lemma': ['octopus', ' ', 'the', 'octopus', 'be', 'a', 'cephalopod', 'of', 'the', 'order', 'Octopoda', '.']}, {'se..."
2,a2.txt.clean,"[{'sentence': 'Cougar The cougar (Puma concolor), also known as puma, mountain lion, mountain cat, catamount or panther, depending on the region, is a mammal of the family Felidae, native to th..."
3,a3.txt.clean,"[{'sentence': 'Koala The koala (Phascolarctos cinereus) is a thickset arboreal marsupial herbivore native to Australia, and the only extant representative of the family Phascolarctidae.', 'lemm..."
4,a4.txt.clean,"[{'sentence': 'Giant_Panda The Giant Panda (Ailuropoda melanoleuca, literally meaning ""cat-foot black-and-white"") is a mammal native to central-western and south western China.', 'lemma': ['gia..."
...,...,...
145,a5.txt.clean,"[{'sentence': 'Anders_Celsius Anders Celsius The observatory of Anders Celsius, from a contemporary engraving.', 'lemma': ['anders_celsius', ' ', 'Anders', 'Celsius', ' ', 'the', 'observatory', ..."
146,a6.txt.clean,"[{'sentence': 'Charles-Augustin_de_Coulomb Charles Augustin de Coulomb (born June 14, 1736, AngoulÃªme, France - died August 23, 1806, Paris, France) was a French physicist.', 'lemma': ['Charle..."
147,a7.txt.clean,"[{'sentence': 'Michael_Faraday Michael Faraday, FRS (September 22, 1791 â August 25, 1867) was an English chemist and physicist (or natural philosopher, in the terminology of that time) who c..."
148,a8.txt.clean,"[{'sentence': 'Amedeo Avogadro Caricature of Amedeo Avogadro Lorenzo Romano Amedeo Carlo Avogadro, Count of Quaregna and Cerreto (August 9, 1776 July 9, 1856) was an Italian savant.', 'lemma': ..."


# Fetching Lemmas Relations

In [53]:
uniqueLemmas = set(corpusLemma)
print('There are:',len(uniqueLemmas),'Unique Lemmas')
uniqueWordRelations = extracRelations(uniqueLemmas)
print('and',len(uniqueWordRelations),'Unique Words Relationtions')

There are: 4372 Unique Lemmas
and 4372 Unique Words Relationtions


In [56]:
# TO show to TA
# print(uniqueWordRelations['duck'])

In [59]:
# TO show to TA
#print(uniqueWordRelations['leopard'])